In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:

from __future__ import absolute_import
from __future__ import print_function
import pylab as pl
import matplotlib.cm as cm
import itertools
import numpy as np
import csv
np.random.seed(1337) # for reproducibility

from keras.layers.noise import GaussianNoise
import keras.models as models
from keras.layers.core import Layer, Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

from keras import backend as K

import cv2
import numpy as np
import scipy
import pandas as pd

Using TensorFlow backend.


In [3]:
#load datasets
DATASETS_PATH = "./MontgomerySet/CXR_png/"

PREFIX = "MCUCXR_"
SUFFIX = "_0.png"

def get_image_filename(index):
    num_zeros = 4-len(str(index))
    img_path = PREFIX + "0"*num_zeros + str(index) + SUFFIX
    return (DATASETS_PATH + img_path)

In [4]:
get_image_filename(1)

'./MontgomerySet/CXR_png/MCUCXR_0001_0.png'

In [5]:
get_image_filename(11)

'./MontgomerySet/CXR_png/MCUCXR_0011_0.png'

In [6]:
LABELS_PATH = "./MontgomerySet/ClinicalReadings/"

label_list = os.listdir(LABELS_PATH)

labels =[]
for fname in label_list:
    with open(os.path.join(LABELS_PATH,fname)) as f:
            lines = f.read().splitlines(True)
            caption =lines[-1]
            labels.append(caption[:-1])
    f.close()
labels = np.asarray(labels)
pd.DataFrame(labels).to_csv("./MontgomerySet/Labels.csv")

In [7]:
BASE_PATH = "./MontgomerySet/"

reader = csv.reader(open(BASE_PATH + 'Labels.csv'))
X =[]
Y =[]

for i,row in enumerate(reader):
    if len(row) > 0:
        filename = get_image_filename(i)
        img_file = cv2.imread(filename)
        if img_file is not None:
            img_file = cv2.cvtColor(img_file, cv2.COLOR_BGR2RGB)
            img_arr = np.asarray(img_file)
            X.append(img_arr)
            Y.append(row[1])
        else:
            print("No file found", filename)

No file found ./MontgomerySet/CXR_png/MCUCXR_0000_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0007_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0009_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0010_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0012_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0014_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0018_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0025_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0032_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0033_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0034_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0036_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0037_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0039_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0050_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0065_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0066_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_006

In [8]:
X = np.asarray(X)
y =np.asarray(Y)

In [9]:
y

array(['extensive cavitary TB smear and culture positive involving LUL lungula and some areas in rt lung as well; pt had previously normal CXR (1Y ago) but positive TST and contact with active case',
       'cavitary nodular infiltrate in RUL; active TB', 'normal',
       'normal',
       'rt pleural effusion with some nodular infiltrate in rt lower lung; there are findings also of previous cardiac surgery; suspicious for active TB.',
       'normal',
       'improved LUL infiltrate and cavity. Active TB on therapy.',
       'peripheral homogeneous infiltrate LUL; active pneumonia likely',
       'normal',
       'bilateral miliary nodules diffusely with RML infiltrate and right pleural effusion; an old calcified granuloma is present behind heart in the LLL; all findings consistent with active TB',
       '', 'normal', 'normal', 'normal',
       'cavitary fibronodular infiltrate RUL. typical appearance of active TB.',
       '',
       'RUL fibrocavitary disease with volume loss and tr

In [10]:
#caption vectorization
token_index ={}

for sample in y:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
max_length = 100
results = np.zeros(shape=(len(y),max_length, max(token_index.values())+ 1))

for i, sample in enumerate(y):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i,j,index] =1